<a href="https://colab.research.google.com/github/chenyu313/TensorFlow-note/blob/main/TensorFlow%E5%86%B3%E7%AD%96%E6%A3%AE%E6%9E%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 使用TensorFlow决策森林构建、训练和评估模型
决策森林(DF)是一系列用于监督分类、回归和排序的机器学习算法。顾名思义，DF使用决策树作为构建块。目前，两种最流行的DF训练算法是随机森林和梯度增强决策树。

TensorFlow Decision Forests (TF-DF)是一个用于训练、评估、解释和推理决策森林模型的库。


## 安装TensorFlow决策森林

In [ ]:
!pip install tensorflow_decision_forests
# 需要Wurlitzer在Colabs中显示详细的训练日志(当在模型构造器中使用verbose=2时)。
!pip install wurlitzer

In [2]:
import tensorflow_decision_forests as tfdf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

In [3]:
# 检查TensorFlow决策森林的版本
print("Found TensorFlow Decision Forests v" + tfdf.__version__)

Found TensorFlow Decision Forests v1.3.0


## 训练随机森林模型
训练、评估、分析和导出一个基于Palmer’s Penguins数据集训练的二元分类随机森林。

注:数据集导出为csv文件，未进行预处理:library(palmerpenguins)；write.csv(penguins，file="penguins.csv"， quote=F, row.names=F)。

### 加载数据集并将其转换为tf.Dataset
这个数据集非常小(300个示例)，并存储为.csv类文件。因此，使用Pandas来加载它。

让我们将数据集组装成一个csv文件(即添加标题)，并加载它:

In [4]:
# 下载数据集
!wget -q https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins.csv -O /tmp/penguins.csv

# 将数据集加载到Pandas Dataframe
dataset_df = pd.read_csv("/tmp/penguins.csv")

# 显示前3个示例
dataset_df.head(3)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007


数据集包含数值特征(例如bill_depth_mm)、分类特征(例如island)和缺失特征的混合。TF-DF原生支持所有这些特征类型(与基于神经网络的模型不同)，因此不需要以one-hot编码、规范化或额外的is_present特征的形式进行预处理。

标签有点不同:Keras指标需要整数。标签(物种)存储为字符串，因此让我们将其转换为整数。

In [5]:
# 将分类标签编码为整数
#
# 细节:
# 如果分类标签表示为字符串，这个阶段是必要的，因为Keras需要整数分类标签.
# 当使用' pd_dataframe_to_tf_dataset '(见下文)时，可以跳过此步骤

# 标签列名称
label = "species"

classes = dataset_df[label].unique().tolist()
print(f"Label classes: {classes}")

dataset_df[label] = dataset_df[label].map(classes.index)

Label classes: ['Adelie', 'Gentoo', 'Chinstrap']


接下来将数据集分成训练和测试两部分:

In [6]:
def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(dataset_df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

250 examples in training, 94 examples for testing.


最后，将pandas数据框(pd.Dataframe)转换为tensorflow数据集(tf.data.Dataset):

In [7]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label)

注意:回想一下pd_dataframe_to_tf_dataset在必要时将字符串标签转换为整数。
如果你想自己创建tf.data.Dataset，有几件事要记住:
* 该学习算法使用单历元数据集，无需随机打乱。
* 批处理大小不会影响训练算法，但较小的值可能会减慢读取数据集的速度。

### 训练模型


In [9]:
# 指定模型
model_1 = tfdf.keras.RandomForestModel(verbose=2)

# 训练模型
model_1.fit(train_ds)

Use 2 thread(s) for training
Use /tmp/tmpc6spxxu8 as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'island': <tf.Tensor 'data:0' shape=(None,) dtype=string>, 'bill_length_mm': <tf.Tensor 'data_1:0' shape=(None,) dtype=float64>, 'bill_depth_mm': <tf.Tensor 'data_2:0' shape=(None,) dtype=float64>, 'flipper_length_mm': <tf.Tensor 'data_3:0' shape=(None,) dtype=float64>, 'body_mass_g': <tf.Tensor 'data_4:0' shape=(None,) dtype=float64>, 'sex': <tf.Tensor 'data_5:0' shape=(None,) dtype=string>, 'year': <tf.Tensor 'data_6:0' shape=(None,) dtype=int64>}
Label: Tensor("data_7:0", shape=(None,), dtype=int64)
Weights: None
Normalized tensor features:
 {'island': SemanticTensor(semantic=<Semantic.CATEGORICAL: 2>, tensor=<tf.Tensor 'data:0' shape=(None,) dtype=string>), 'bill_length_mm': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast:0' shape=(None,) dtype=float32>), 'bill_depth_mm': SemanticTensor(semantic=<Semantic.NUMERIC

[INFO 23-04-24 10:42:16.2123 UTC kernel.cc:773] Start Yggdrasil model training
[INFO 23-04-24 10:42:16.2124 UTC kernel.cc:774] Collect training examples
[INFO 23-04-24 10:42:16.2125 UTC kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 23-04-24 10:42:16.2136 UTC kernel.cc:393] Number of batches: 1
[INFO 23-04-24 10:42:16.2136 UTC kernel.cc:394] Number of examples: 250
[INFO 23-04-24 10:42:16.2140 UTC kernel.cc:794] Training dataset:
Number of records: 250
Number of columns: 8

Number of columns by type:
	NUMERICAL: 5 (62.5%)
	CATEGORICAL: 3 (37.5%)

Columns:

NUMERICAL: 5 (62.5%)
	1: "bill_depth_mm" NUMERICAL num-nas:2 (0.8%) mean:17.2375 min:13.1 m

Model trained in 0:00:00.266062
Compiling model...


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


### 备注
* 没有指定输入特性。因此，除了标签之外，所有列都将用作输入特征。模型使用的特征显示在训练日志和model.summary()中。
* DF消耗原生的数值、分类、分类集特征和缺失值。数值特征不需要规范化。分类字符串值不需要在字典中编码。
* 没有指定训练超参数。因此，将使用默认的超参数。默认超参数在大多数情况下提供合理的结果。
* 在匹配之前对模型调用compile是可选的。Compile可用于提供额外的评估指标。
* 训练算法不需要验证数据集。如果提供了验证数据集，它将仅用于显示指标。
* 将verbose参数调整为RandomForestModel，以控制显示的训练日志数量。设置verbose=0以隐藏大部分日志。设置verbose=2以显示所有日志。

注意:一个分类集特征是由一组分类值组成的(而一个分类只有一个值)。稍后将给出更多细节和示例。

## 评价模型
用测试集评价模型

In [11]:
model_1.compile(metrics=["accuracy"])
evaluation = model_1.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

1/1 [==============================] - 0s 142ms/step - loss: 0.0000e+00 - accuracy: 1.0000

loss: 0.0000
accuracy: 1.0000


将模型导出为SavedModel格式以便以后重用，例如TensorFlow Serving。

In [12]:
model_1.save("/tmp/my_saved_model")

## 模型可视化
绘制决策树并跟踪第一个分支有助于了解决策森林。在某些情况下，绘制模型甚至可以用于调试。

由于它们的训练方式不同，一些模型的规划比其他模型更有趣。由于在训练过程中注入的噪声和树的深度，绘制随机森林的信息量不如绘制CART或梯度增强树的第一棵树。

In [13]:
tfdf.model_plotter.plot_model_in_colab(model_1, tree_idx=0, max_depth=3)

【说明】：左边的根节点包含第一个条件(bill_depth_mm >= 16.40)、示例数(250)和标签分布(红-蓝-绿条)。

计算值为true的示例bill_depth_mm >= 16.40分支到绿色路径。其他的是红色路径的分支。

节点越深，它们就越纯粹，即标签分布偏向于类的子集。

## 模型结构和特征重要性
模型的整体结构用.summary()显示。你会看到:
* Type（类型）:用于训练模型的学习算法(在我们的例子中是随机森林)。
* Task（任务）:模型解决的问题(在我们的例子中是分类)。
* Input Features（输入特征）:模型的输入特征。
* Variable Importance（变量重要性）:模型中每个特征重要性的不同度量。
* Out-of-bag evaluation（包外评价）:对模型的包外评价。这是一种廉价而有效的替代交叉验证的方法。
* {树、节点}的数目和其他指标:关于决策林结构的统计数据。

备注:摘要的内容取决于学习算法(例如，Out-of-bag仅适用于Random Forest)和超参数(例如，在超参数中可以禁用平均降低精度的变量重要性)。

In [14]:
model_1.summary()

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (7):
	bill_depth_mm
	bill_length_mm
	body_mass_g
	flipper_length_mm
	island
	sex
	year

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1. "flipper_length_mm"  0.440100 ################
    2.    "bill_length_mm"  0.389837 ############
    3.     "bill_depth_mm"  0.341018 ########
    4.            "island"  0.307496 #####
    5.       "body_mass_g"  0.258674 ##
    6.               "sex"  0.234169 
    7.              "year"  0.231934 

Variable Importance: NUM_AS_ROOT:
    1. "flipper_length_mm" 158.000000 ################
    2.     "bill_depth_mm" 85.000000 ########
    3.    "bill_length_mm" 40.000000 ###
    4.    

## 参考
https://www.tensorflow.org/decision_forests/tutorials/beginner_colab#model_structure_and_feature_importance